In [1]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.applications import imagenet_utils
from keras.preprocessing.image import *
from keras.utils import multi_gpu_model

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from glob import glob
from tqdm import tqdm
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import *

import multiprocessing
from multiprocessing.dummy import Pool
from pprint import pprint

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))

Using TensorFlow backend.


In [2]:
batch_size = 128
# preprocess_func = xception.preprocess_input
# MODEL = Xception

preprocess_func = nasnet.preprocess_input
MODEL = NASNetLarge

width = 334
height = 334
classes = ['正常', '吊经', '擦洞', '跳花', '毛洞', '织稀', '扎洞', '缺经', '毛斑', '其他']
n_classes = len(classes)

In [3]:
idg2 = ImageDataGenerator()
gen_valid = idg2.flow_from_directory('crop_valid_64/', target_size=(height, width), shuffle=False, 
                                     interpolation='lanczos', classes=classes, batch_size=batch_size)
y_valid = (gen_valid.classes != 0).astype(int)

Found 190008 images belonging to 10 classes.


In [4]:
input_tensor = Input((height, width, 3))

x = Lambda(preprocess_func)(input_tensor)
base_model = MODEL(include_top=False, weights='imagenet', input_tensor=x)

x = GlobalAvgPool2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(n_classes, activation='softmax')(x)
model = Model(input_tensor, x)

In [5]:
parallel_model = multi_gpu_model(model, gpus=n_gpus)

# 统计每个图片切割了多少个

In [6]:
fnames = gen_valid.filenames
len(fnames)

190008

In [7]:
from collections import Counter

def get_main(a):
    return a[:a.rfind('_', 0, a.rfind('_'))]

Counter(map(get_main, fnames))

Counter({'正常/J01_2018.06.13 13_24_39': 936,
         '正常/J01_2018.06.13 13_27_54': 936,
         '正常/J01_2018.06.13 14_41_29': 936,
         '正常/J01_2018.06.13 14_44_12': 936,
         '正常/J01_2018.06.13 14_47_09': 936,
         '正常/J01_2018.06.16 08_47_33': 936,
         '正常/J01_2018.06.16 08_49_15': 936,
         '正常/J01_2018.06.16 09_18_33': 936,
         '正常/J01_2018.06.16 09_25_55': 936,
         '正常/J01_2018.06.16 09_41_45': 936,
         '正常/J01_2018.06.16 10_22_48': 936,
         '正常/J01_2018.06.16 10_36_08': 936,
         '正常/J01_2018.06.17 09_51_21': 936,
         '正常/J01_2018.06.17 10_20_27': 936,
         '正常/J01_2018.06.17 13_23_04': 936,
         '正常/J01_2018.06.17 13_48_27': 936,
         '正常/J01_2018.06.17 13_53_53': 936,
         '正常/J01_2018.06.17 14_09_56': 936,
         '正常/J01_2018.06.17 14_19_57': 936,
         '正常/J01_2018.06.17 14_48_02': 936,
         '正常/J01_2018.06.17 15_34_09': 936,
         '正常/J01_2018.06.17 15_38_47': 936,
         '正常/J01_2018.06.19 09_3

# 进行预测

In [8]:
glob('models/nasnet_0726-0.000001*')

['models/nasnet_0726-0.000001-35-0.621782.h5',
 'models/nasnet_0726-0.000001-31-0.617965.h5',
 'models/nasnet_0726-0.000001-32-0.618060.h5',
 'models/nasnet_0726-0.000001-34-0.619480.h5',
 'models/nasnet_0726-0.000001-33-0.618600.h5']

In [9]:
path = 'models/nasnet_0726-0.000001-35-0.621782.h5'
model.load_weights(path)

In [10]:
y_pred = parallel_model.predict_generator(gen_valid, verbose=1, workers=multiprocessing.cpu_count())

y_pred2 = 1 - y_pred[:,0]
y_pred3 = 1 - (np.argmax(y_pred, axis=-1) == 0)
pprint(roc_auc_score(y_valid, y_pred2))
pprint(accuracy_score(y_valid, y_pred3))
pprint(confusion_matrix(y_valid > 0.5, y_pred3))

1485/1485 [==============================] - 2018s 1s/step
0.6279998856214699
0.6878341964548861
array([[122284,   1268],
       [ 58046,   8410]])


# 计算 AUC

In [21]:
n1 = 936
y_valid = (gen_valid.classes[::n1] != 0).astype(int)
y_pred2 = y_pred.reshape((-1, n1, 10))
y_pred3 = list(map(lambda x:Counter(x.argmax(axis=-1))[0] / 936, y_pred2))
print('AUC:', roc_auc_score(y_valid, 1-np.array(y_pred3)))
print('AUC:', roc_auc_score(y_valid, 1-y_pred2.mean(axis=1)[:,0]))

AUC: 0.9127720870678617
AUC: 0.8923388817755016


In [22]:
for path in glob('models/nasnet_0726-0.000001*'):
    print(path)
    model.load_weights(path)
    y_pred = parallel_model.predict_generator(gen_valid, verbose=1, workers=multiprocessing.cpu_count())
    
    n1 = 936
    y_valid = (gen_valid.classes[::n1] != 0).astype(int)
    y_pred2 = y_pred.reshape((-1, n1, 10))
    y_pred3 = list(map(lambda x:Counter(x.argmax(axis=-1))[0] / 936, y_pred2))
    print('AUC:', roc_auc_score(y_valid, 1-np.array(y_pred3)))
    print('AUC:', roc_auc_score(y_valid, 1-y_pred2.mean(axis=1)[:,0]))

models/nasnet_0726-0.000001-35-0.621782.h5
1485/1485 [==============================] - 1974s 1s/step
AUC: 0.9127720870678617
AUC: 0.8923388817755016
models/nasnet_0726-0.000001-31-0.617965.h5
1485/1485 [==============================] - 1991s 1s/step
AUC: 0.9207212974818609
AUC: 0.9036491677336747
models/nasnet_0726-0.000001-32-0.618060.h5
1485/1485 [==============================] - 1994s 1s/step
AUC: 0.922801963294921
AUC: 0.9022620571916347
models/nasnet_0726-0.000001-34-0.619480.h5
1485/1485 [==============================] - 1981s 1s/step
AUC: 0.9145326504481434
AUC: 0.8935125906956893
models/nasnet_0726-0.000001-33-0.618600.h5
1485/1485 [==============================] - 1993s 1s/step
AUC: 0.9204545454545454
AUC: 0.8981007255655142
